In [9]:
"""
현재까지 만들어진 이미지 객체들
hero : 본체
bullet : 총알
enemy : 적군
===============================
bulList : 총알 저장 리스트
enemList: 적군 저장 리스트
"""


import pygame
import random
from datetime import datetime      #경과시간을 위해
import time as t

"""
1. 게임 초기화
"""

pygame.init()

"""
2. 게임 창 옵션 설정
"""


#크기 설정
size = [800,1000]                         
screen = pygame.display.set_mode(size)

 #제목 설정
title = "Save Earth"                    
pygame.display.set_caption(title)        



"""
3. 게임 내 필요한 설정
"""

#시계 만들기
clock = pygame.time.Clock()              


backgroundImg = pygame.image.load(r'image\background1.jpg')
backgroundImg = pygame.transform.scale(backgroundImg, (800, 1000))

logoImg = pygame.image.load(r'image\logo.png').convert_alpha()
logoImg = pygame.transform.scale(logoImg, (600, 500))


#클래스 생성
class createObj:
    #생성자
    def __init__(self):
        self.x = 0
        self.y = 0
        self.move = 0
        self.kill = 0
        self.delay = 0
        self.tic=0
    #캐릭터 입력
    def insertImg(self, addr):
        if addr[-3:] == 'png':
            self.obj = pygame.image.load(addr).convert_alpha()
        else:
            self.obj = pygame.image.load(addr)
        #캐릭터 x길이, y길이 받아오기
        self.oX, self.oY = self.obj.get_size()
        
    #사진 사이즈 설정
    def setSize(self, oX, oY):
        self.obj = pygame.transform.scale(self.obj, (oX, oY))
        #캐릭터 x길이, y길이 받아오기
        self.oX, self.oY = self.obj.get_size()
    
    #blit
    def show(self):
        screen.blit(self.obj, (self.x, self.y))

        
        
#충돌 탐지 함수
"""
충돌 탐지 함수 crashDetec ()
서로 다른 이미지들이 충돌하면 True값을 반환하는 함수입니다.

ex) a=crashDetec(hero,enemy)
>>>print(a)
>>>True
"""
def crashDetec(vic,perp):
    if vic.x-perp.oX+5 < perp.x and perp.x < vic.x+vic.oX-5:
        if vic.y-perp.oY+5 < perp.y and perp.y < vic.y+vic.oY-5:
            return True
        else:
            return False
    else:
        return False

    
"""
충돌 좌표 함수 crashCoor()
서로 다른 이미지들이 충돌하면 충돌 좌표를 반환하는 함수입니다.
"""
def crashCoor(vic,perp):
    if vic.x-perp.oX+5 < perp.x and perp.x < vic.x+vic.oX-5:
        if vic.y-perp.oY+5 < perp.y and perp.y < vic.y+vic.oY-5:
            return perp.x, perp.y

        
        
        

"""
게임 종료 함수 end_game(point,shootDown,missed,sub_time,game_over,clear)
"""

def end_game(point,shootDown,missed,sub_time,game_over,clear=False):
    """
    0. 게임 종료 
    """
    point += sub_time*5

    while game_over == 1:
        #FPS 설정 초당 반응 수
        clock.tick(60)

        """
        0-1. 입력 감지
        """
        #스페이스가 눌리면 게임 시작
        for event in pygame.event.get():
            #x버튼 누르면 종료
            if event.type == pygame.QUIT:
                game_over=0

        """
        0-2. 그리기
        """
        #텍스트 폰트 설정
        font = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",70)
        font2 = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",30)
        font3 = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",40)

        """
        글자 생성
        """
        if clear == True:     #게임 클리어 시
            end_textG = font.render("GAME CLEAR",True, (255,0,0))
        else:
            end_textG = font.render("GAME OVER",True, (255,0,0))

        end_textS = font2.render(f"Shoot Down",True, (0,128,0))
        end_textSP = font2.render(f"{shootDown}  x  100",True, (0,0,255))
        end_textM = font2.render(f"Missed",True, (0,128,0))
        end_textMP = font2.render(f"{missed}  x  -50",True, (0,0,255))
        end_textT = font2.render(f"Play Time",True, (0,128,0))
        end_textTP = font2.render(f"{sub_time}  x  5",True, (0,0,255))

        end_textP = font3.render(f"Point",True, (0,128,0))
        end_textPP = font3.render(f"{point}p",True, (0,0,255))

        bar = font.render("|",True, (255,255,255))
        topBar = font.render("===========",True, (255,255,255))
        bottomBar = font.render("------------",True, (255,255,255))

        bottomBar2 = font2.render("------------------------",True, (255,255,255))
        #텍스트 표시
        screen.blit(topBar, (207,250))
        screen.blit(end_textG, (207,300))
        screen.blit(end_textS, (220,400))
        screen.blit(end_textSP, (450,400))
        screen.blit(end_textM, (220,450))
        screen.blit(end_textMP, (450,450))
        screen.blit(end_textT, (220,500))
        screen.blit(end_textTP, (450,500))
        screen.blit(bottomBar2, (220,540))
        screen.blit(end_textP, (220,600))
        screen.blit(end_textPP, (450,600))

        screen.blit(bar,(180,270))
        screen.blit(bar,(620,270)) 
        screen.blit(bar,(180,320))
        screen.blit(bar,(620,320))
        screen.blit(bar,(180,370))
        screen.blit(bar,(620,370))
        screen.blit(bar,(180,420))
        screen.blit(bar,(620,420))
        screen.blit(bar,(180,470))
        screen.blit(bar,(620,470))
        screen.blit(bar,(180,520))
        screen.blit(bar,(620,520))
        screen.blit(bar,(180,570))
        screen.blit(bar,(620,570))
        screen.blit(bar,(180,620))
        screen.blit(bar,(620,620))
        screen.blit(bottomBar,(199,650))
        
        """
        0-3. 업데이트
        """
        #screen 상태 업데이트
        pygame.display.flip()
    
        
    
    
"""
=======================
캐릭터 객체 생성 및 설정
"""
hero = createObj()
#캐릭터 경로 설정
hero.insertImg(r'image\hero.png')
#캐릭터 크기 설정
hero.setSize(60,90)
#캐릭터 위치 설정
hero.x = round(size[0]/2 - hero.oX/2)
hero.y = size[1] - 50 - hero.oY
hero.move = 10
"""
================
"""


"""
=======================
보스 1 객체 생성 및 설정
"""
first_boss = createObj()
#캐릭터 경로 설정
first_boss.insertImg(r'image\boss1.png')
#캐릭터 크기 설정
first_boss.setSize(700,300)
#캐릭터 위치 설정
first_boss.x = round(size[0]/2 - first_boss.oX/2)
first_boss.y = -200
first_boss.move = 1
"""
================
"""



#생명 리스트
lifeList=[]
#체력 추가 리스트
healPlusList = []
#체력 아이템 생성 됐는가 여부
healCheck=False





#적기 격추 = shootDown
shootDown=0

#적기 놓침 = missed
missed=0

#포인트값
#point = (shootDown*100)-(missed-50)
point=0

#무적 시간 체크
tic=0


#무적 시간
invinTime = 2



"""
초기화면
"""

"""
초기화면. 변수 설정
"""
#버튼 누르면 startB값이 0이되어 종료
startB = 1

#텍스트 색
text_index = 0


#startB가 1이 아니라면 while문 종료
while startB == 1:
    #FPS 설정 초당 반응 수
    clock.tick(60)
    
    """
    초기화면. 입력 감지
    """
    #스페이스가 눌리면 게임 시작
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == 32 or event.key == 107:            #press on space bar
                startB = 0
    
    """
    초기화면. 값의 변화
    """
    text_index+=1
    if text_index == 36000:
        text_index=0
    
    

            
    
    """
    초기화면. 그리기
    """
    #배경사진 넣기
    screen.blit(backgroundImg,(0,0))
    #로고사진 넣기
    screen.blit(logoImg,(100,70))
    
    #텍스트 폰트 설정
    font = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",20)
    #스페이스 누르라는 텍스트
    if text_index // 9 % 2 == 0:
        start_text = font.render("Press SPACE BAR to start game!",True, (255,255,255))
    else:
        start_text = font.render("Press SPACE BAR to start game!",True, (128,128,128))
    #텍스트 표시
    screen.blit(start_text, (250,round(size[1]/2)+300))
    
    """
    초기화면. 업데이트
    """
    
    #screen 상태 업데이트
    pygame.display.flip()  




"""
Stage 1. 시작
"""

"""
Stage 1. 변수 설정
"""
#시작 현재 시간 불러오기
start_time = datetime.now()

# 변수 설정
#버튼 누르면 end_s1값이 0이되어 종료
end_s1 = 1
#k : 총알 속도 연관 변수
k=0

###초기화 설정###
#이동 값(bool) 초기화
leftMove=False
rightMove=False
topMove = False
bottomMove = False
#슈팅 여부
shot = False

#총알 리스트
bulList_s1=[]
#적군 리스트
enemList_s1=[]
#타격효과 리스트
hitList_s1=[]
#타격 효과 화면 표시 인덱스
hit_idx_s1=0
#피격 효과 리스트
redList_s1=[]
#처치 효과 리스트
popList_s1=[]






#game_over가 1이 되면 게임오버 창이 나오도록
game_over_s1 = 0
# 스테이지 클리어시 True로 바뀌면 다음 스테이지 진행
# 게임 오버시 게임 종료
stage_clear_s1 = False


#무적 상태 유무
invin_s1 = False



"""
==================================
캐릭터 체력 화면 표시 정의 구간
"""
lifeNum=5             #캐릭터 HP
for i in range(lifeNum):
    life = createObj()
    life.insertImg(r'image\life.png')
    life.setSize(40,40)
    life.x = 100 + i*50
    life.y = 0
    lifeList.append(life)

"""
==================================
"""

#end_s1가 1이 아니라면 while문 종료
while end_s1 == 1:
    """
    Stage 1. FPS 설정
    """
    #FPS 설정 초당 반응 수
    clock.tick(60)                       

    """
    Stage 1. 각종 입력 감지
    """
    
    #입력 감지
    
    for event in pygame.event.get():
        #x버튼 누르면 종료
        if event.type == pygame.QUIT:
            end_s1=0
            
        """
        =====================================================
        wasd or 방향키 입력 or 슈팅 입력(space bar or k) 감지
        """
        #KeyDown되면 활성화
        if event.type == pygame.KEYDOWN:
            if event.key == 97 or event.key == 1073741904:     #press on a
                leftMove = True
            if event.key == 100 or event.key == 1073741903:    #press on d
                rightMove = True
            if event.key == 115 or event.key == 1073741905:    #press on s
                bottomMove = True
            if event.key == 119 or event.key == 1073741906:    #press on w
                topMove = True
            if event.key == 32 or event.key == 107:            #press on space bar
                shot = True
                k=0
        #KeyUp 되면 비활성화
        elif event.type == pygame.KEYUP:
            if event.key == 97 or event.key == 1073741904:     #press off a
                leftMove = False
            if event.key == 100 or event.key == 1073741903:    #press on d
                rightMove = False
            if event.key == 115 or event.key == 1073741905:    #press off s
                bottomMove = False
            if event.key == 119 or event.key == 1073741906:    #press off w
                 topMove = False
            if event.key == 32 or event.key == 107:            #press off space bar
                shot = False 
        """
        =====================================================
        """
    
    
    """
    Stage 1. 입력과 시간에 따른 변화
    """
    #게임 진행 시 현재 시간 불러오기
    now_time = datetime.now()
    #진행되는 시간 구하기
    sub_time = round((now_time - start_time).total_seconds(),1)
    """
    ============================
    캐릭터 이동에 관한 부분 정의
    """
    #좌측 이동
    if leftMove == True:
        hero.x -= hero.move
        #화면에 벗어나지 않도록
        if hero.x <= 0:
            hero.x = 0
    #우측 이동
    if rightMove == True:
        hero.x += hero.move
        #화면에 벗어나지 않도록
        if hero.x >=size[0]-hero.oX:
            hero.x = size[0]-hero.oX
    #위로 이동
    if topMove == True:
        hero.y -= hero.move
        #화면에 벗어나지 않도록
        if hero.y <= 0 :
            hero.y = 0
    #아래로 이동
    if bottomMove == True:
        hero.y += hero.move
        #화면에 벗어나지 않도록
        if hero.y >= size[1]-hero.oY:
            hero.y = size[1]-hero.oY
    """
    ============================
    """

    """
    ============================================================
    캐릭터가 총알 쏠 때 
    bul_num의 변수 값의 k % n 중 n값이 작아질 수록 총알 빨라짐.
    """
    bul_num = k%10
    if shot == True and bul_num == 0:
        bullet = createObj()
        bullet.insertImg(r'image\bullet.png')
        bullet.setSize(30,50)
        bullet.x = round(hero.x + hero.oX/2 - bullet.oX/2)
        bullet.y = hero.y - bullet.oY - 10
        bullet.move = 20
        bulList_s1.append(bullet)
    k+=1
    
    # 생성된 총알 움직이기
    del_bulList=[]
    for i,v in enumerate(bulList_s1):
        v.y -= v.move
        # 총알이 화면을 벗어나면 제거
        if v.y <= -v.oY:
            del_bulList.append(i)
    for i in del_bulList:
        del bulList_s1[i]
    """
    ============================================================
    """
    
    """
    =====================
    체력 증가 아이템 생성
    heal4Kill : 몇 킬 마다 힐 아이템 나오게 할지 정하는 변수
    """
    #지금은 해당 shootDown이 되면 화면을 장악함, move도 안됨.
    #그래서 만들면 True False로 구분해야 할 듯
    heal4Kill = 3
    if shootDown != 0 and shootDown % heal4Kill == 0 and healCheck==False:
        healPlus = createObj()
        healPlus.insertImg(r'image\healPlus.png')
        healPlus.setSize(70,70)
        healPlus.x = random.randrange(0, size[0]-healPlus.oX, healPlus.oX)
        healPlus.y = 70
        healPlus.move = 1
        healPlusList.append(healPlus)
        shootDown_check = shootDown
        healCheck=True
    if healCheck == True and shootDown != shootDown_check:
        healCheck=False
        
    # 생성된 체력 증가 아이템 움직이기
    del_healPlusList=[]
    for i,v in enumerate(healPlusList):
        v.y += v.move
        # 체력 증가 아이템이 화면을 벗어나면 제거
        if v.y <= -v.oY:
            del_healPlusList.append(i)
    for i in del_healPlusList:
        del healPlusList[i]
    
    """
    =====================
    """
    
    
    
    """
    =========================================================================================
    랜덤 적기 생성
    enem_num_index의 값은 0~1사이 값입니다.
    1로 설정하면 적군이 하나도 나오지 않고 1에 가까워질수록 적군이 나오는 빈도수가 줄어듭니다.
    """
    enem_num_index = 0.97
    if random.random() > enem_num_index:
        enemy = createObj()
        enemy.insertImg(r'image\enemy.png')
        enemy.setSize(70,80)
        #randrange구간을 설정해줘서 좌우 겹치는 적기 없도록 설정
        enemy.x = random.randrange(0, size[0]-enemy.oX, enemy.oX )
        enemy.y = 70
        enemy.move = 2
        enemList_s1.append(enemy)
    
    #생성된 적기 움직이기
    del_enemList=[]
    for i,v in enumerate(enemList_s1):
        v.y += v.move
        # 적기가 화면을 벗어나면 제거
        if v.y > size[1]+v.oY:
            del_enemList.append(i)
    for i in del_enemList:
        del enemList_s1[i]
        missed+=1             #적기 놓치면 증가
        point-=50             #적기 놓치면 -50
        
    """
    =========================================================================================
    """
    
    
    
    
    
    
    
    """
    ==========================================================
    적군과 총알이 충돌하면 일어나는 이벤트를 정의한 구간입니다.
    deathNum의 값을 통해서 적군의 HP를 설정할 수 있습니다.
    """
    deathNum=3        #적군 HP
    cd_del_enemList=[]        #피격 적군 삭제 리스트
    cd_del_bulList=[]         #타격 총알 삭제 리스트
    del_hitList=[]            #피격 시 이벤트 삭제 리스트
    del_popList=[]            #격추 시 이벤트 삭제 리스트
    for i,enem in enumerate(enemList_s1):
        for j,bul in enumerate(bulList_s1):
            if crashDetec(enem,bul):             #타격 시 
                
                """
                ----------------------
                타격 시 타격 효과 생성
                """
                hit = createObj()
                hit.insertImg(r'image\hit.png')
                hit.setSize(80,80)
                eX,eY = crashCoor(enem,bul)
                hit.x = eX - round(hit.oX/2) + 20
                hit.y = eY - round(hit.oY/2)
                hit.tic = t.time()
                hitList_s1.append(hit)
                """
                ----------------------
                """
                
                #총알 삭제 리스트에 추가
                enem.kill+=1
                cd_del_bulList.append(j)
                
                #격추 판정되면 적군 제거 리스트 추가
                if enem.kill==deathNum:
                    """
                    ---------------------------
                    격추 판정 시 격추 효과 생성
                    """
                    pop=createObj()
                    pop.insertImg(r'image\pop.png')
                    pop.setSize(70,70)
                    pop.x = eX - round(pop.oX/2) + 20
                    pop.y = eY - round(pop.oY/2)
                    pop.tic = t.time()
                    popList_s1.append(pop)
                    """
                    ---------------------------
                    """
                    
                    cd_del_enemList.append(i)
                    enem.kill=0
                    
                    
    """
    ---------------------------------
    타격 효과 삭제
    hit_time으로 인해 유지 시간 정함.
    """
    hit_time = 0.1
    for i in hitList_s1:
        hit_toc=t.time()
        if round(i.tic,1) + hit_time <= round(hit_toc,1):
            del hitList_s1[0]
    """
    ---------------------------------
    """
    
    """
    ---------------------------------
    격추 효과 삭제
    pop_time으로 인해 유지 시간 정함.
    """
    pop_time = 0.5
    for i in popList_s1:
        pop_toc=t.time()
        if round(i.tic,1)+pop_time <= round(pop_toc,1):
            del popList_s1[0]
    """
    ---------------------------------
    """

    
    """
    -----------------------------
    피격, 타격으로 인한 객체 삭제
    """    
    cd_del_enemList=list(set(cd_del_enemList))  #중복 피격 방지
    cd_del_bulList=list(set(cd_del_bulList))    #중복 타격 방지
    
    cd_del_enemList.reverse()
    cd_del_bulList.reverse()

    #삭제 코드
    for i in cd_del_enemList:
        del enemList_s1[i]          #적기 제거
        shootDown += 1           #적군 격추 시 증가
        point += 100             #적군 격추 시 포인트 100 증가
        
    for i in cd_del_bulList:
        del bulList_s1[i]           #총알 제거
        
    """
    -----------------------------
    """
        
    """
    ==========================================================
    """
    
    
    """
    ===========================================================
    캐릭터와 적군이 충돌할 때 일어나는 이벤트를 정의한 구간입니다.
    lifeNum 변수를 통해서 본체의 HP를 설정할 수 있습니다.
    """
    
    #invin_s1 == True면 무적 False면 무적 아님.
    del_lifeList=[]
    for i,enem in enumerate(enemList_s1):
        if crashDetec(hero,enem) and  invin_s1==False:
            tic = t.time()
            invin_s1 = True
            hero.kill += 1
            del lifeList[-1]        #목숨 하나 제거
            
            #무적상태일 때 레드 스크린 효과 적용
            redScreen = createObj()
            redScreen.insertImg(r'image\redScreen.png')
            redScreen.setSize(800,1000)
            hit.x = 0
            hit.y = 0
            redList_s1.append(redScreen)
            
            if hero.kill == lifeNum:
                end_s1=0               #게임 종료
                game_over_s1=1          #게임 오버
    
    toc = t.time()
    if round(tic+invinTime) == round(toc):          #무적 끝나면 invin_s1 = False로 만들어서 무적 풀고
        if invin_s1 == True:
            del redList_s1[-1]                 #레드 스크린 효과 종료
            invin_s1 = False
        
    """
    ===========================================================
    """
    
    """
    ===============================================================
    캐릭터가 healPlus를 먹으면 일어나는 이벤트를 정의한 구간입니다.
    """   

    for heal in healPlusList:
        if crashDetec(hero,heal):
            del healPlusList[0]
            if lifeNum-hero.kill <=9:
                life = createObj()
                life.insertImg(r'image\life.png')
                life.setSize(40,40)
                life.x = 100 + (lifeNum-hero.kill)*50
                life.y = 0
                lifeList.append(life)
                lifeNum+=1
    """
    ===============================================================
    """
        
    
    """
    Stage 1. 그리기
    """
    
    #화면 색 채우기
#     screen.fill(color)
    screen.blit(backgroundImg,(0,0))
    #캐릭터 띄우기
    hero.show()
    
    #총알 띄우기
    for v in bulList_s1:
        v.show()
    
    #적기 띄우기
    for v in enemList_s1:
        v.show()
        
    for v in hitList_s1:
        v.show()
        
    for v in redList_s1:
        v.show()
        
    for v in popList_s1:
        v.show()
        
    for v in lifeList:
        v.show()

    for v in healPlusList:
        v.show()
    
    
    
    """
    ================
    텍스트 표시 절차
    """
    #1. 폰트 설정(폰트 주소, 폰트 크기)
    font = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",20)
    font2 = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",30)
    
    #2. surface 생성(텍스트, 안티앨리어싱, 폰트 색(rgb))
    #shootDown 값과 missed값 표시
    text_shoot = font.render(f"Shoot Down : {shootDown}",True, (255,255,255))
    text_missed = font.render(f"Missed : {missed}",True, (255,255,255))
    
    text_time = font.render(f"Time : {sub_time}",True,(255,255,255))
    text_life = font2.render("Life : ",True,(255,255,255))
    text_stage = font2.render("Stage 1",True,(255,255,255))
    
    
    
    #3. surface 표시(표시할 이미지, 위치)
    screen.blit(text_stage, (620,5))
    screen.blit(text_shoot, (620,40))
    screen.blit(text_missed, (620,60))
    screen.blit(text_time, (620,80))
    screen.blit(text_life, (10,5))
    
    
    """
    ================
    """

    """
    Stage 1. 업데이트
    """
    #screen 상태 업데이트
    pygame.display.flip()
    if game_over_s1 == 1:
        end_game(point,shootDown,missed,sub_time,game_over_s1)
    if shootDown == 10:
        end_s1=0
        stage_clear_s1 = True

        
        
"""
=====================================================================================================================
=====================================================================================================================
=====================================================================================================================
=====================================================================================================================
"""        
             
        
        
        
        
        
"""
BOSS 1. 시작
"""

"""
BOSS 1. 변수 설정
"""

check_start_time = t.time()

# 변수 설정
#버튼 누르면 end_b1값이 0이되어 종료
end_b1 = 1
#k : 총알 속도 연관 변수
k=0

###초기화 설정###
#이동 값(bool) 초기화
leftMove=False
rightMove=False
topMove = False
bottomMove = False
#슈팅 여부
shot = False

#총알 리스트
bulList_b1=[]
#적군 리스트
enemList_b1=[]
#타격효과 리스트
hitList_b1=[]
#효과 화면 표시
hit_idx_b1=0
#처치 효과 리스트
popList_b1=[]
#피격 효과 리스트
redList_b1=[]

#game_over가 1이 되면 게임오버 창이 나오도록
game_over_b1 = 0
# 스테이지 클리어시 True로 바뀌면 다음 스테이지 진행
# 게임 오버시 게임 종료
stage_clear_b1 = False

#무적 상태 유무
invin_b1 = False

#적군 HP
deathNum=5        


"""
==================================
캐릭터 체력 화면 표시 정의 구간
"""
# hero.kill=0           #보스 진입 시 HP 초기화
# lifeNum=5             #캐릭터 HP
for i in range(lifeNum-hero.kill):
    life = createObj()
    life.insertImg(r'image\life.png')
    life.setSize(40,40)
    life.x = 100 + i*50
    life.y = 0
    lifeList.append(life)

"""
==================================
"""

#end_b1가 1이 아니라면 while문 종료
while end_b1 == 1 and stage_clear_s1:
    """
    BOSS 1. FPS 설정
    """
    #FPS 설정 초당 반응 수
    clock.tick(60)                       

    """
    BOSS 1. 각종 입력 감지
    """
    
    #입력 감지
    
    for event in pygame.event.get():
        #x버튼 누르면 종료
        if event.type == pygame.QUIT:
            end_b1=0
            
        """
        =====================================================
        wasd or 방향키 입력 or 슈팅 입력(space bar or k) 감지
        """
        #KeyDown되면 활성화
        if event.type == pygame.KEYDOWN:
            if event.key == 97 or event.key == 1073741904:     #press on a
                leftMove = True
            if event.key == 100 or event.key == 1073741903:    #press on d
                rightMove = True
            if event.key == 115 or event.key == 1073741905:    #press on s
                bottomMove = True
            if event.key == 119 or event.key == 1073741906:    #press on w
                topMove = True
            if event.key == 32 or event.key == 107:            #press on space bar
                shot = True
                k=0
        #KeyUp 되면 비활성화
        elif event.type == pygame.KEYUP:
            if event.key == 97 or event.key == 1073741904:     #press off a
                leftMove = False
            if event.key == 100 or event.key == 1073741903:    #press on d
                rightMove = False
            if event.key == 115 or event.key == 1073741905:    #press off s
                bottomMove = False
            if event.key == 119 or event.key == 1073741906:    #press off w
                 topMove = False
            if event.key == 32 or event.key == 107:            #press off space bar
                shot = False 
        """
        =====================================================
        """
    
    
    """
    BOSS 1. 입력과 시간에 따른 변화
    """
    #게임 진행 시 현재 시간 불러오기
    now_time = datetime.now()
    #진행되는 시간 구하기
    sub_time = round((now_time - start_time).total_seconds(),1)
    """
    ============================
    캐릭터 이동에 관한 부분 정의
    """
    #좌측 이동
    if leftMove == True:
        hero.x -= hero.move
        #화면에 벗어나지 않도록
        if hero.x <= 0:
            hero.x = 0
    #우측 이동
    if rightMove == True:
        hero.x += hero.move
        #화면에 벗어나지 않도록
        if hero.x >=size[0]-hero.oX:
            hero.x = size[0]-hero.oX
    #위로 이동
    if topMove == True:
        hero.y -= hero.move
        #화면에 벗어나지 않도록
        if hero.y <= 0 :
            hero.y = 0
    #아래로 이동
    if bottomMove == True:
        hero.y += hero.move
        #화면에 벗어나지 않도록
        if hero.y >= size[1]-hero.oY:
            hero.y = size[1]-hero.oY
    """
    ============================
    """

    """
    ============================================================
    캐릭터가 총알 쏠 때 
    bul_num의 변수 값의 k % n 중 n값이 작아질 수록 총알 빨라짐.
    """
    bul_num = k%10
    if shot == True and bul_num == 0:
        bullet = createObj()
        bullet.insertImg(r'image\bullet.png')
        bullet.setSize(30,50)
        bullet.x = round(hero.x + hero.oX/2 - bullet.oX/2)
        bullet.y = hero.y - bullet.oY - 10
        bullet.move = 20
        bulList_b1.append(bullet)
    k+=1
    
    # 생성된 운석 움직이기
    del_bulList=[]
    for i,v in enumerate(bulList_b1):
        v.y -= v.move
        #총알이 화면을 벗어나면 제거
        if v.y <= -v.oY:
            del_bulList.append(i)
    for i in del_bulList:
        del bulList_b1[i]
    """
    ============================================================
    """
    
    """
    =====================
    체력 증가 아이템 생성
    heal4Kill : 몇 킬 마다 힐 아이템 나오게 할지 정하는 변수
    """
    #지금은 해당 shootDown이 되면 화면을 장악함, move도 안됨.
    #그래서 만들면 True False로 구분해야 할 듯
    heal4Kill = 3
    if shootDown != 0 and shootDown % heal4Kill == 0 and healCheck==False:
        healPlus = createObj()
        healPlus.insertImg(r'image\healPlus.png')
        healPlus.setSize(70,70)
        healPlus.x = random.randrange(0, size[0]-healPlus.oX, healPlus.oX)
        healPlus.y = 70
        healPlus.move = 1
        healPlusList.append(healPlus)
        shootDown_check = shootDown
        healCheck=True
    if healCheck == True and shootDown != shootDown_check:
        healCheck=False
        
    # 생성된 체력 증가 아이템 움직이기
    del_healPlusList=[]
    for i,v in enumerate(healPlusList):
        v.y += v.move
        # 체력 증가 아이템이 화면을 벗어나면 제거
        if v.y <= -v.oY:
            del_healPlusList.append(i)
    for i in del_healPlusList:
        del healPlusList[i]
    
    """
    =====================
    """
    
    
    
    """
    ==========================================================================================
    랜덤 미사일 생성
    enem_num_index의 값은 0~1사이 값입니다.
    1로 설정하면 적군이 하나도 나오지 않고 1에 가까워질수록 적군이 나오는 빈도수가 줄어듭니다.
    
    check_now_time 변수와 check_start_time 변수를 이용해서 
    보스 스테이지 진입 후 2초 뒤에 운석들이 생기도록 설정
    """
    
    check_now_time = t.time()
    if round(check_start_time+2) < round(check_now_time):
        enem_num_index = 0.97
        if random.random() > enem_num_index:
            enemy = createObj()
            enemy.insertImg(r'image\enemy1.png')
            enemy.setSize(40,80)
            #randrange구간을 설정해줘서 좌우 겹치는 적기 없도록 설정
            enemy.x = random.randrange(0, size[0]-enemy.oX, enemy.oX )
            enemy.y = 70
            enemy.move = 2
            enemList_b1.append(enemy)

        # 생성된 운석 움직이기
        del_enemList=[]
        for i,v in enumerate(enemList_b1):
            v.y += v.move
            # 운석이 화면을 벗어나면 제거
            if v.y > size[1]+v.oY:
                del_enemList.append(i)
        for i in del_enemList:
            del enemList_b1[i]
            missed+=1             #적기 놓치면 증가
            point-=50             #적기 놓치면 -50
        
    """
    ==========================================================================================
    """
    
    
    """
    ===========================================================
    보스와 총알이 충돌하면 일어나는 이벤트를 정의한 구간입니다.
    """
    cd_del_bulList=[]         #타격 총알 삭제 리스트
    first_boss_HP = 200        #보스 HP 현재는 20이지만 200으로 설정할 예정
    
    for i,bul in enumerate(bulList_b1):
        if crashDetec(first_boss,bul):
            
            
            """
            =================
            타격 시 타격 효과 생성
            """
            hit = createObj()
            hit.insertImg(r'image\hit.png')
            hit.setSize(50,50)
            eX,eY = crashCoor(first_boss,bul)
            hit.x = eX - round(hit.oX/2) + 20
            hit.y = eY - round(hit.oY/2) - 20
            hit.tic = t.time()
            hitList_b1.append(hit)
            """
            =================
            """
            
            #Boss 피격
            first_boss.kill+=1
            
            #총알 삭제 리스트에 추가
            cd_del_bulList.append(i)
            
            if first_boss.kill == first_boss_HP:
                point+=10000
                end_b1=0
    
    """
    ===========================================================
    """
    
    
    
    
    """
    ==========================================================
    미사일과 총알이 충돌하면 일어나는 이벤트를 정의한 구간입니다.
    deathNum의 값을 통해서 적군의 HP를 설정할 수 있습니다.
    """
    
    cd_del_enemList=[]        #피격 적군 삭제 리스트
    del_hitList=[]         #피격 시 이펙트 삭제 리스트
    for i,enem in enumerate(enemList_b1):
        for j,bul in enumerate(bulList_b1):
            if crashDetec(enem,bul):             #타격 시 
                
                """
                ----------------------
                타격 시 타격 효과 생성
                """
                eX,eY = crashCoor(enem,bul)
                hit = createObj()
                hit.insertImg(r'image\hit.png')
                hit.setSize(80,80)
                hit.x = eX - round(hit.oX/2) + 20
                hit.y = eY - round(hit.oY/2)
                hit.tic = t.time()
                hitList_b1.append(hit)
                print(1)
                """
                ----------------------
                """
                
                #총알 삭제 리스트에 추가
                cd_del_bulList.append(j)
                #적군 피격 카운팅
                enem.kill+=1
                
                #격추 판정되면 적군 제거 리스트 추가 
                if enem.kill==deathNum:
                    """
                    ---------------------------
                    격추 판정 시 격추 효과 생성
                    """
                    pop=createObj()
                    pop.insertImg(r'image\pop.png')
                    pop.setSize(70,70)
                    pop.x = eX - round(pop.oX/2) + 20
                    pop.y = eY - round(pop.oY/2)
                    pop.tic = t.time()
                    popList_b1.append(pop)
                    """
                    ---------------------------
                    """
                    cd_del_enemList.append(i)
                    enem.kill=0
                    
                    
#     #조건을 충족하면
#     if hit_idx_b1 % 13 == 0:
#         #이펙트 삭제 리스트 추가
#         for k,v in enumerate(hitList_b1):
#             del_hitList.append(k)
#     hit_idx_b1 += 1
    
    """
    ---------------------------------
    타격 효과 삭제
    hit_time으로 인해 유지 시간 정함.
    """
    hit_time = 0.1
    for i in hitList_b1:
        hit_toc=t.time()
        if round(i.tic,1) + hit_time <= round(hit_toc,1):
            del hitList_b1[0]
    """
    ---------------------------------
    """
    
    """
    ---------------------------------
    격추 효과 삭제
    pop_time으로 인해 유지 시간 정함.
    """
    pop_time = 0.5
    for i in popList_b1:
        pop_toc=t.time()
        if round(i.tic,1)+pop_time <= round(pop_toc,1):
            del popList_b1[0]
    """
    ---------------------------------
    """
            
    cd_del_enemList=list(set(cd_del_enemList))  #중복 피격 방지
    cd_del_bulList=list(set(cd_del_bulList))    #중복 타격 방지
    
    cd_del_enemList.reverse()
    cd_del_bulList.reverse()
    #삭제 코드
    for i in cd_del_enemList:
        del enemList_b1[i]          #적기 제거
        shootDown += 1           #적군 격추 시 증가
        point += 100             #적군 격추 시 포인트 100 증가
        
    for i in cd_del_bulList:
        del bulList_b1[i]           #총알 제거
        
    """
    ==========================================================
    """
    
    
    """
    ===========================================================
    캐릭터와 적군이 충돌할 때 일어나는 이벤트를 정의한 구간입니다.
    lifeNum 변수를 통해서 본체의 HP를 설정할 수 있습니다.
    """
    
    #invin_b1 == True면 무적 False면 무적 아님.
    for i,enem in enumerate(enemList_b1):
        if (crashDetec(hero,enem) and  invin_b1==False) or (crashDetec(hero,first_boss) and invin_b1==False):
            tic = t.time()
            invin_b1 = True
            hero.kill += 1
            del lifeList[-1]        #목숨 하나 제거
            
            #무적상태일 때 레드 스크린 효과 적용
            redScreen = createObj()
            redScreen.insertImg(r'image\redScreen.png')
            redScreen.setSize(800,1000)
            hit.x = 0
            hit.y = 0
            redList_b1.append(redScreen)
            
            
            if hero.kill == lifeNum:
                end_b1=0               #게임 종료
                game_over_b1=1          #게임 오버
                

    #invinTime = 2
    toc = t.time()
    if round(tic+invinTime) == round(toc):          #무적 끝나면 invin = False로 만들어서 무적 풀고
        if invin_b1 == True:
            del redList_b1[-1]                 #레드 스크린 효과 종료
            invin_b1 = False
        
    """
    ===========================================================
    """
    
    """
    ===============================================================
    캐릭터가 healPlus를 먹으면 일어나는 이벤트를 정의한 구간입니다.
    """   

    for heal in healPlusList:
        if crashDetec(hero,heal):
            del healPlusList[0]
            if lifeNum-hero.kill <=9:
                life = createObj()
                life.insertImg(r'image\life.png')
                life.setSize(40,40)
                life.x = 100 + (lifeNum-hero.kill)*50
                life.y = 0
                lifeList.append(life)
                lifeNum+=1
    """
    ===============================================================
    """

    
    """
    BOSS 1. 그리기
    """
    
    #화면 색 채우기
#     screen.fill(color)
    screen.blit(backgroundImg,(0,0))
    #캐릭터 띄우기
    hero.show()
    
    #보스 띄우기
    #움직이면서 나오는 효과를 줌
    first_boss.show()
    if first_boss.y < 0:
        first_boss.y += first_boss.move
    
    for v in redList_b1:
        v.show()
    
    #총알 띄우기
    for v in bulList_b1:
        v.show()
    
    #적기 띄우기
    for v in enemList_b1:
        v.show()
    #타격 이펙트
    for v in hitList_b1:
        v.show()
    # 피격 이펙트    
    for v in popList_b1:
        v.show()
        
    for v in lifeList:
        v.show()
        
    for v in healPlusList:
        v.show()
        
    
    
    """
    ================
    텍스트 표시 절차
    """
    #1. 폰트 설정(폰트 주소, 폰트 크기)
    font = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",20)
    font2 = pygame.font.Font(r"font\경기천년제목V_Bold.ttf",30)
    
    #2. surface 생성(텍스트, 안티앨리어싱, 폰트 색(rgb))
    #shootDown 값과 missed값 표시
    text_shoot = font.render(f"Shoot Down : {shootDown}",True, (255,255,255))
    text_missed = font.render(f"Missed : {missed}",True, (255,255,255))
    
    text_time = font.render(f"Time : {sub_time}",True,(255,255,255))
    text_life = font2.render("Life : ",True,(255,255,255))
    text_stage = font2.render("BOSS",True,(255,0,0))
    
    
    
    #3. surface 표시(표시할 이미지, 위치)
    screen.blit(text_stage, (620,5))
    screen.blit(text_shoot, (620,40))
    screen.blit(text_missed, (620,60))
    screen.blit(text_time, (620,80))
    screen.blit(text_life, (10,5))
    
    
    """
    ================
    """

    """
    BOSS 1. 업데이트
    """
    #screen 상태 업데이트
    pygame.display.flip()
    if game_over_b1 == 1:
        end_game(point,shootDown,missed,sub_time,game_over_b1)
    if shootDown == 1000:
        end_b1=0
        stage_clear_b1 = True


pygame.quit()

1
1
1
1
1
1
1
1
1
1


In [ ]:
pip install pygame

In [ ]:
import os
print(os.getcwd())